In [ ]:
import transformers
print(transformers.__version__)

#4.5.1

In [ ]:
!pip install import_ipynb

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, AutoModel
import import_ipynb
from datasets import load_dataset
from os import listdir
from os.path import isfile, join

In [ ]:
path1 = "dataset/..."
path2 = "dataset/..."
#onlyfiles = [f for f in listdir(path1) if isfile(join(path1, f))]
all_files = []

In [ ]:
def concanListStrings(list1, list2):
    list3 = []
    myLen1 = len(list1)
    if myLen1 != len(list2):
        print("Length - error")
    for idx in range(0, myLen1):
        list3.append(list1[idx] + " " + list2[idx])
    return list3

In [ ]:
#Generate the datasets with the different fields.
def generate_datasets(df):
    sentencesTitle = df.title.values
    sentencesCont = df.docCont.values
    sentencesTitleCont = concanListStrings(sentencesTitle, sentencesCont)

    return sentencesTitleCont

In [ ]:
def get_dataset_from_huggingface(dataset):
    def join(example):
        example['text'] = ' '.join(str(example['title']) + str(example['docCont']))
        return example

    return dataset.map(join, num_proc=8, remove_columns=['link', 'title', 'docCont'])

In [ ]:
def group_texts(block_size:int):
    def func(examples):
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])

        total_length = (total_length // block_size) * block_size

        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result['labels'] = result['input_ids'].copy()
        return result
    return func

In [ ]:
def tokenize(tokenizer):
    def func(df):
        return tokenizer(df['text'])
    return func

In [ ]:
model_checkpoint = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
model.cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
import datasets
from datasets import load_from_disk

my_path = "dataset/..."
lm_dataset = load_from_disk(my_path)

In [ ]:
print(lm_dataset)

In [ ]:
transformers.logging.set_verbosity_info()

In [ ]:
training_args = TrainingArguments(
output_dir="./models/ROBERTA/",
evaluation_strategy='epoch',
save_steps = 10000,
num_train_epochs = 2,
learning_rate = 2e-5,
logging_steps = 10000,
disable_tqdm = True,
weight_decay = 0.1,
adam_beta2  = 0.98,
adam_epsilon = 1e-6,
warmup_ratio  = 0.1,
gradient_accumulation_steps = 2)

trainer = Trainer(
model=model,
args=training_args,
train_dataset=lm_dataset['train'],
eval_dataset=lm_dataset['test'],
data_collator=data_collator)

In [ ]:
print(f"Started training")
trainer.train("./models/ROBERTA/checkpoint-30000")

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {np.exp(eval_results['eval_loss']):.2f}")

In [ ]:
print(f"Saving the model")
model.save_pretrained('./models/BERT_SERP/mlm_model_serp_en.pt')